In [2]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('data/rssi.csv')

In [218]:
# Test with smaller df

# df.head(50000).to_csv('data/rssi_50k.csv')
# df = pd.read_csv('data/rssi_50k.csv')

In [15]:
# create TourID

df['AreaNumerDif'] = df['AreaNumber'] - df['AreaNumber'].shift(-1) # find new stations
df.loc[df['AreaNumerDif']>0, 'AreaNumerDif'] = 1 # steps between stations always 1 or -1, depending on train direction
df.loc[df['AreaNumerDif']<0, 'AreaNumerDif'] = -1
df['AreaNumerDif'] = df['AreaNumerDif'].replace(0, np.nan) # nan where no change of station
df['AreaNumerDif'] = df['AreaNumerDif'].fillna(method='backfill') # fill with direction of next station change
df['AreaNumerDif'] = df['AreaNumerDif'].fillna(method='ffill') # fill last rows of dataset

df['AreaNumerDifDif'] = df['AreaNumerDif'] - df['AreaNumerDif'].shift(-1) # find shifts of direction
df.loc[df['AreaNumerDifDif']!=0, 'AreaNumerDifDif'] = df.loc[df['AreaNumerDifDif']!=0, 'ID'] # set row id where direction changes
df['AreaNumerDifDif'] = df['AreaNumerDifDif'].replace(0, np.nan) # nan, where no change of direction
df['AreaNumerDifDif'] = df['AreaNumerDifDif'].fillna(method='backfill') # fill with TourID of next TourID
df.rename(columns = {'AreaNumerDifDif':'TourID'}, inplace = True)

In [16]:
# create TrackSegment

df['TrackSegment'] = df['PositionNoLeap'] / 1000
df['TrackSegment'] = df['TrackSegment'].astype(int) # cut last digits

In [17]:
# test
# df.to_csv('data/rssi_50k_test.csv')

In [19]:
# Create SNR

df['A1_TotalTelDif'] = df['A1_TotalTel'] - df['A1_TotalTel'].shift(1)
df['A1_ValidTelDif'] = df['A1_ValidTel'] - df['A1_ValidTel'].shift(1)
df['SNR'] = df['A1_ValidTelDif'] / df['A1_TotalTelDif']

In [20]:
df_segments = df[['Latitude', 'Longitude', 'A2_RSSI', 'SNR', 'TourID', 'TrackSegment']]
aggregations = {
    'Latitude': 'mean',
    'Longitude': 'mean',
    'A2_RSSI': 'min',
    'SNR': 'min',
    'TourID': 'min'}
df_segments = df_segments.groupby(['TrackSegment', 'TourID']).agg(aggregations)

In [21]:
df_segments

Latitude  Longitude  A2_RSSI       SNR      TourID
TrackSegment TourID                                                         
97           196777.0    47.304858   8.049463     0.07  0.988525    196777.0
             234047.0    47.305277   8.049646     1.88  1.000000    234047.0
             348416.0    47.305015   8.049532     0.07  0.000000    348416.0
             625056.0    47.305286   8.049649     2.26  1.000000    625056.0
             666568.0    47.304984   8.049518     0.93  0.000000    666568.0
...                            ...        ...      ...       ...         ...
428          29581190.0  47.240285   8.189582     2.06  0.999666  29581190.0
             29609237.0  47.240287   8.189581     2.06  0.999801  29609237.0
             29891348.0  47.240293   8.189579     2.30  0.999068  29891348.0
             30169798.0  47.240296   8.189577     2.26  1.000000  30169798.0
             30409183.0  47.240301   8.189576     2.30  0.999285  30409183.0

[1091876 rows x 5 columns]

In [23]:
df_segments.to_csv('data/rssi_segmented.csv')